In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs

In [2]:
triples_data = pd.read_csv('dataset/train_triples.tsv', sep='\t', header=None, names=['e1_id', 'e2_id', 'rel_id', 'e1_label', 'e2_label', 'rel_label'])

In [3]:
triples_data

,e1_id,e2_id,rel_id,e1_label,e2_label,rel_label
0,Q18394838,Q482994,P31,Trick,album,instance of
1,Q1324135,Q1563,P131,Floridita,Havana,located in the administrative territorial entity
2,Q68155,Q65453,P47,Schwyz,Illgau,shares border with
3,Q257065,Q2259451,P106,Nancy Marchand,stage actor,occupation
4,Q1663240,Q11424,P31,Spun,film,instance of
5,Q130469,Q889,P17,Mazar-i-Sharif,Afghanistan,country
6,Q164119,Q33999,P106,Mark Wahlberg,actor,occupation
7,Q453074,Q158852,P106,Brian Setzer,conductor,occupation
8,Q550428,Q33231,P106,Jeff Wall,photographer,occupation
9,Q356033,Q33525,P279,adenocarcinoma,carcinoma,subclass of


In [4]:
def get_list_sents(title): 
    if title == 'N/A':
        return []
    
    response = requests.get('https://en.wikipedia.org/wiki/' + title)
    html = response.text
    soup = bs(html, 'html.parser')
    sent_list = []
    
    for text in soup.findAll('p'):
        paragraph = text.get_text().replace('\n', ' ')
        paragraph = re.sub('[[0-9]*]', ' ', paragraph)
        sentences = paragraph.split('. ')
        sent_list += sentences
    sent_list = list(filter(None, sent_list))
    
    return sent_list

In [5]:
def get_wikipedia_title(wikidata_id):
    url = 'https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=sitelinks&ids=' + wikidata_id + '&sitefilter=enwiki'
    r = requests.get(url)
    result = r.json()
    try:
        title = result['entities'][wikidata_id]['sitelinks']['enwiki']['title']
    except KeyError:
        title = 'N/A'
    return title

In [39]:
#filtered_sents = []
len_triples = len(triples_data['e1_id'])

for i in range(37224, len_triples):
    print(str(i) + '/' + str(len_triples))
    
    e1_label = triples_data['e1_label'][i]
    e2_label = triples_data['e2_label'][i]
    e1_id = triples_data['e1_id'][i]
    e2_id = triples_data['e2_id'][i]
    rel_id = triples_data['rel_id'][i]
    rel_label = triples_data['rel_label'][i]
    
    e1 = get_wikipedia_title(e1_id)
    e2 = get_wikipedia_title(e2_id)
    sents = get_list_sents(e1) + get_list_sents(e2)
    
    for sent in sents:
        if e1_label in e2_label: # i.e. e1: asia, e2: southeast asia
            if e2_label in sent:
                sent_temp = sent.replace(e2_label, '<<<<_ENTITY2_>>>>')
                if e1_label in sent_temp:
                    filtered_sents.append((e1_id, e2_id, rel_id, e1_label, e2_label, rel_label, sent.strip()))
        elif e2_label in e1_label: # i.e. e1: southeast asia, e2: asia
            if e1_label in sent:
                sent_temp = sent.replace(e1_label, '<<<<_ENTITY1_>>>>')
                if e2_label in sent_temp:
                    filtered_sents.append((e1_id, e2_id, rel_id, e1_label, e2_label, rel_label, sent.strip()))
        else:
            if e1_label in sent and e2_label in sent:
                filtered_sents.append((e1_id, e2_id, rel_id, e1_label, e2_label, rel_label, sent.strip()))

37224/38725
37225/38725
37226/38725
37227/38725
37228/38725
37229/38725
37230/38725
37231/38725
37232/38725
37233/38725
37234/38725
37235/38725
37236/38725
37237/38725
37238/38725
37239/38725
37240/38725
37241/38725
37242/38725
37243/38725
37244/38725
37245/38725
37246/38725
37247/38725
37248/38725
37249/38725
37250/38725
37251/38725
37252/38725
37253/38725
37254/38725
37255/38725
37256/38725
37257/38725
37258/38725
37259/38725
37260/38725
37261/38725
37262/38725
37263/38725
37264/38725
37265/38725
37266/38725
37267/38725
37268/38725
37269/38725
37270/38725
37271/38725
37272/38725
37273/38725
37274/38725
37275/38725
37276/38725
37277/38725
37278/38725
37279/38725
37280/38725
37281/38725
37282/38725
37283/38725
37284/38725
37285/38725
37286/38725
37287/38725
37288/38725
37289/38725
37290/38725
37291/38725
37292/38725
37293/38725
37294/38725
37295/38725
37296/38725
37297/38725
37298/38725
37299/38725
37300/38725
37301/38725
37302/38725
37303/38725
37304/38725
37305/38725
37306/38725
3730

37907/38725
37908/38725
37909/38725
37910/38725
37911/38725
37912/38725
37913/38725
37914/38725
37915/38725
37916/38725
37917/38725
37918/38725
37919/38725
37920/38725
37921/38725
37922/38725
37923/38725
37924/38725
37925/38725
37926/38725
37927/38725
37928/38725
37929/38725
37930/38725
37931/38725
37932/38725
37933/38725
37934/38725
37935/38725
37936/38725
37937/38725
37938/38725
37939/38725
37940/38725
37941/38725
37942/38725
37943/38725
37944/38725
37945/38725
37946/38725
37947/38725
37948/38725
37949/38725
37950/38725
37951/38725
37952/38725
37953/38725
37954/38725
37955/38725
37956/38725
37957/38725
37958/38725
37959/38725
37960/38725
37961/38725
37962/38725
37963/38725
37964/38725
37965/38725
37966/38725
37967/38725
37968/38725
37969/38725
37970/38725
37971/38725
37972/38725
37973/38725
37974/38725
37975/38725
37976/38725
37977/38725
37978/38725
37979/38725
37980/38725
37981/38725
37982/38725
37983/38725
37984/38725
37985/38725
37986/38725
37987/38725
37988/38725
37989/38725
3799

38590/38725
38591/38725
38592/38725
38593/38725
38594/38725
38595/38725
38596/38725
38597/38725
38598/38725
38599/38725
38600/38725
38601/38725
38602/38725
38603/38725
38604/38725
38605/38725
38606/38725
38607/38725
38608/38725
38609/38725
38610/38725
38611/38725
38612/38725
38613/38725
38614/38725
38615/38725
38616/38725
38617/38725
38618/38725
38619/38725
38620/38725
38621/38725
38622/38725
38623/38725
38624/38725
38625/38725
38626/38725
38627/38725
38628/38725
38629/38725
38630/38725
38631/38725
38632/38725
38633/38725
38634/38725
38635/38725
38636/38725
38637/38725
38638/38725
38639/38725
38640/38725
38641/38725
38642/38725
38643/38725
38644/38725
38645/38725
38646/38725
38647/38725
38648/38725
38649/38725
38650/38725
38651/38725
38652/38725
38653/38725
38654/38725
38655/38725
38656/38725
38657/38725
38658/38725
38659/38725
38660/38725
38661/38725
38662/38725
38663/38725
38664/38725
38665/38725
38666/38725
38667/38725
38668/38725
38669/38725
38670/38725
38671/38725
38672/38725
3867

In [40]:
len(filtered_sents)

97451

In [41]:
filtered_sents[0], filtered_sents[-1]

(('Q18394838',
  'Q482994',
  'P31',
  'Trick',
  'album',
  'instance of',
  'Trick is the second solo album by Kele Okereke, the lead singer of British indie rock band Bloc Party'),
 ('Q4165206',
  'Q484282',
  'P131',
  'Berea',
  'Box Butte County',
  'located in the administrative territorial entity',
  'Berea is about 10 miles (16\xa0km) northwest of Alliance, the county seat of Box Butte County'))

In [38]:
triples_data['e1_label'][37224], triples_data['e2_label'][37224]

('Arthur Koestler', 'writer')

In [42]:
with open('dataset/train.tsv', 'w') as f:
    for item in filtered_sents:
        f.write(item[0] + '\t' + item[1] + '\t' + item[2] + '\t' + item[3] + '\t' + item[4] +
                '\t' + item[5] + '\t' + item[6] + '\n')
f.closed

True